# Self-Driving Cab

## Simulación de un Taxi Autónomo

Diseñemos una simulación de un taxi autónomo. El objetivo principal es demostrar, en un entorno simplificado, cómo se pueden utilizar técnicas de aprendizaje por refuerzo para desarrollar un enfoque eficiente y seguro para abordar este problema.

El trabajo del Smartcab es recoger al pasajero en una ubicación y dejarlo en otra. Aquí hay algunas cosas que nos encantaría que nuestro Smartcab se encargara:

- Dejar al pasajero en la ubicación correcta.
- Ahorrar tiempo al pasajero tomando el tiempo mínimo posible para dejarlo.
- Cuidar la seguridad del pasajero y las normas de tráfico.

Hay diferentes aspectos que deben considerarse aquí al modelar una solución de aprendizaje por refuerzo para este problema: recompensas, estados y acciones.


## Recompensa y Penalización

Dado que el agente (el conductor imaginario) está motivado por las recompensas y aprenderá a controlar el taxi mediante experiencias de prueba en el entorno, necesitamos decidir las recompensas y/o penalizaciones y su magnitud en consecuencia. Aquí hay algunos puntos a considerar:

* El agente debería recibir una recompensa positiva alta por una entrega exitosa porque este comportamiento es altamente deseado.
* El agente debería ser penalizado si intenta dejar al pasajero en ubicaciones incorrectas.
* El agente debería recibir una ligera penalización negativa por no llegar al destino después de cada paso de tiempo. "Ligera" negativa porque preferiríamos que nuestro agente llegue tarde en lugar de hacer movimientos incorrectos tratando de llegar al destino lo más rápido posible.


## State space

En Aprendizaje por Refuerzo, el agente encuentra un estado y luego toma una acción de acuerdo con el estado en el que se encuentra.

El Espacio de Estados es el conjunto de todas las situaciones posibles en las que nuestro taxi podría encontrarse. El estado debe contener información útil que el agente necesita para tomar la acción correcta.

Digamos que tenemos un área de entrenamiento para nuestro Smartcab donde lo estamos enseñando a transportar personas en un estacionamiento a cuatro ubicaciones diferentes (R, G, Y, B):


<img src="./img/Reinforcement_Learning_Taxi_Env.png" alt="drawing" width="500"/>

## Espacio de Estados

Supongamos que Smartcab es el único vehículo en este estacionamiento. Podemos dividir el estacionamiento en una cuadrícula de 5x5, lo que nos da 25 ubicaciones de taxi posibles. Estas 25 ubicaciones son una parte de nuestro espacio de estados. Observa que el estado de ubicación actual de nuestro taxi es la coordenada (3, 1).

También notarás que hay cuatro (4) ubicaciones donde podemos recoger y dejar a un pasajero: R, G, Y, B o [(0,0), (0,4), (4,0), (4,3)] en coordenadas (fila, columna). Nuestro pasajero ilustrado está en la ubicación Y y desea ir a la ubicación R.

Cuando también tenemos en cuenta un (1) estado adicional del pasajero de estar dentro del taxi, podemos tomar todas las combinaciones de ubicaciones de pasajeros y ubicaciones de destino para llegar a un número total de estados para nuestro entorno de taxi; hay cuatro (4) destinos y cinco (4 + 1) ubicaciones de pasajeros.


In [1]:
# Cálculo del espacio de estados total del entorno Taxi
# El entorno es una cuadrícula de 5x5 con 4 ubicaciones especiales (R, G, Y, B)
# Fórmula: posiciones_taxi * ubicaciones_pasajero * ubicaciones_destino
# - 5x5 = 25 posiciones posibles para el taxi en la cuadrícula
# - 5 ubicaciones posibles del pasajero: en R, G, Y, B, o dentro del taxi (4+1)
# - 4 ubicaciones de destino posibles: R, G, Y, o B
# Total de estados posibles:
5*5*5*4  # = 500 estados únicos posibles

500

## Action space

## Acciones en el Entorno del Taxi

El agente encuentra uno de los 500 estados y toma una acción. La acción en nuestro caso puede ser moverse en una dirección o decidir recoger/dejar a un pasajero.

En otras palabras, tenemos seis acciones posibles:
1. sur
2. norte
3. este
4. oeste
5. recoger
6. dejar


## Espacio de Acciones

Este es el espacio de acciones: el conjunto de todas las acciones que nuestro agente puede tomar en un estado dado.

Observarás en la ilustración anterior que el taxi no puede realizar ciertas acciones en ciertos estados debido a las paredes. En el código del entorno, simplemente proporcionaremos una penalización de -1 por cada colisión con una pared y el taxi no se moverá en absoluto. Esto simplemente acumulará penalizaciones, haciendo que el taxi considere ir alrededor de la pared.


## Implementation

In [2]:
# =============================================================================
# IMPORTACIÓN DE LIBRERÍAS NECESARIAS
# =============================================================================

# Gymnasium: Librería moderna de entornos de Reinforcement Learning (antes OpenAI Gym)
# Es la versión actualizada y mantenida de Gym, compatible con NumPy moderno
import gymnasium as gym

# Time: Para controlar pausas y tiempos de espera en las animaciones
import time

# Matplotlib (comentado): Para visualizar gráficas de rendimiento
# import matplotlib.pyplot as plt

In [3]:
# =============================================================================
# CREACIÓN DEL ENTORNO TAXI
# =============================================================================

# gym.make() crea una instancia del entorno "Taxi-v3"
# - "Taxi-v3": Entorno predefinido de un taxi que debe recoger y dejar pasajeros
# - render_mode="ansi": Modo de renderizado en texto ASCII para visualizar en consola
env = gym.make("Taxi-v3", render_mode="ansi")

# env.reset() reinicia el entorno a un estado inicial aleatorio
# Devuelve dos valores:
# - state: El estado inicial (un número entre 0-499)
# - info: Diccionario con información adicional del entorno
state, info = env.reset()

# env.render() genera una representación visual del estado actual
# En este caso, muestra la cuadrícula del taxi en formato ASCII
print(env.render())

# Líneas comentadas para posibles usos futuros:
# time.sleep(10)  # Pausar la ejecución 10 segundos
# env.close()     # Cerrar el entorno y liberar recursos

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




## Interfaz del Entorno Gymnasium

La interfaz central de Gymnasium es `env`, que es la interfaz de entorno unificada. Los siguientes son los métodos de `env` que serían bastante útiles para nosotros:

* `env.reset`: Reinicia el entorno y devuelve un estado inicial aleatorio y un diccionario de información.
* `env.step(action)`: Avanza el entorno en un paso de tiempo. Devuelve
    - `observation`: Observaciones del entorno
    - `reward`: Si tu acción fue beneficiosa o no
    - `terminated`: Indica si hemos recogido y dejado con éxito a un pasajero, también llamado un episodio
    - `truncated`: Si el episodio se trunca debido a un límite de tiempo
    - `info`: Información adicional como rendimiento y latencia con fines de depuración
* `env.render`: Renderiza un fotograma del entorno (útil para visualizar el entorno)


## Reto

**Hay 4 ubicaciones (etiquetadas con diferentes letras), y nuestro trabajo es recoger al pasajero en una ubicación y dejarlo en otra. Recibimos +20 puntos por una entrega exitosa y perdemos 1 punto por cada paso de tiempo que toma. También hay una penalización de 10 puntos por acciones de recogida y entrega ilegales.**


In [4]:
# =============================================================================
# EXPLORACIÓN DEL ESPACIO DE ESTADOS Y ACCIONES
# =============================================================================

# Reiniciamos el entorno para obtener un nuevo estado aleatorio
state, info = env.reset()

# Mostramos cómo se ve el entorno en este nuevo estado
print(env.render())

# env.action_space: Define todas las acciones posibles que puede tomar el agente
# En Taxi-v3 hay 6 acciones: 0=sur, 1=norte, 2=este, 3=oeste, 4=recoger, 5=dejar
print("Action Space {}".format(env.action_space))

# env.observation_space: Define todos los estados posibles del entorno
# En Taxi-v3 hay 500 estados únicos (Discrete(500))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


Action Space Discrete(6)
State Space Discrete(500)


* El cuadrado lleno representa el taxi, que es amarillo sin pasajero y verde con pasajero.
* El símbolo "|" representa una pared que el taxi no puede cruzar.
* R, G, Y, B son las posibles ubicaciones de recogida y destino. La letra azul representa la ubicación actual de recogida del pasajero, y la letra morada es el destino actual.


Según lo verificado por las impresiones, tenemos un Espacio de Acciones de tamaño 6 y un Espacio de Estados de tamaño 500. Como verás, nuestro algoritmo de aprendizaje por refuerzo no necesitará más información que estas dos cosas. Todo lo que necesitamos es una forma de identificar un estado de manera única asignando un número único a cada estado posible, y el aprendizaje por refuerzo aprende a elegir un número de acción de 0 a 5 donde:

0 = sur
1 = norte
2 = este
3 = oeste
4 = recoger
5 = dejar


Recuerda que los 500 estados corresponden a una codificación de la ubicación del taxi, la ubicación del pasajero y la ubicación de destino.

El Aprendizaje por Refuerzo aprenderá un mapeo de estados a la acción óptima a realizar en ese estado mediante la exploración, es decir, el agente explora el entorno y toma acciones basadas en las recompensas definidas en el entorno.

La acción óptima para cada estado es la acción que tiene la recompensa acumulativa a largo plazo más alta.


De hecho, podemos tomar nuestra ilustración anterior, codificar su estado y dárselo al entorno para que lo renderice en Gym. Recuerda que tenemos el taxi en la fila 3, columna 1, nuestro pasajero está en la ubicación 2 y nuestro destino está en la ubicación 0. Usando el método de codificación de estado Taxi-v3, podemos hacer lo siguiente:


In [5]:
# =============================================================================
# CODIFICACIÓN MANUAL DE UN ESTADO ESPECÍFICO
# =============================================================================

# env.unwrapped.encode() convierte coordenadas legibles en un número de estado (0-499)
# Parámetros:
# - taxi_row: Fila donde está el taxi (0-4)
# - taxi_col: Columna donde está el taxi (0-4)
# - passenger_index: Ubicación del pasajero (0=R, 1=G, 2=Y, 3=B, 4=en taxi)
# - destination_index: Destino deseado (0=R, 1=G, 2=Y, 3=B)

# Creamos un estado donde:
# - Taxi en fila 1, columna 3
# - Pasajero en G (índice 1)
# - Destino en Y (índice 2)
state = env.unwrapped.encode(1, 3, 1, 2)
print("State:", state)

# Establecemos manualmente este estado en el entorno
# env.unwrapped.s permite acceder al estado interno del entorno
env.unwrapped.s = state

# Visualizamos el estado que acabamos de configurar
print(env.render())

State: 166
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




Estamos utilizando las coordenadas de nuestra ilustración para generar un número correspondiente a un estado entre 0 y 499, lo cual resulta ser 324 para el estado de nuestra ilustración.

Luego podemos establecer manualmente el estado del entorno utilizando env.unwrapped.s con ese número codificado. Puedes experimentar con los números y verás que el taxi, el pasajero y el destino se mueven.


## The Reward Table

Cuando se crea el entorno Taxi, también se crea una tabla de recompensas inicial llamada `P`. Podemos pensar en ella como una matriz que tiene el número de estados como filas y el número de acciones como columnas.

Dado que cada estado está en esta matriz, podemos ver los valores de recompensa predeterminados asignados al estado de nuestra ilustración:


In [6]:
# =============================================================================
# CONFIGURACIÓN DE UN ESTADO ESPECÍFICO PARA ANÁLISIS
# =============================================================================

# Codificamos el estado del ejemplo de la ilustración:
# - Taxi en fila 3, columna 1
# - Pasajero en G (índice 1)
# - Destino en R (índice 0)
state = env.unwrapped.encode(3, 1, 1, 0)
print("State:", state)  # Debería ser el estado 324

# Establecemos este estado manualmente en el entorno
env.unwrapped.s = state

# Renderizamos para ver cómo se ve visualmente
print(env.render())

State: 324
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




In [7]:
# =============================================================================
# TABLA DE TRANSICIONES DEL ENTORNO (P)
# =============================================================================

# env.unwrapped.P es un diccionario que contiene toda la dinámica del entorno
# Para cada estado, muestra qué pasa al tomar cada acción
# Estructura: {acción: [(probabilidad, próximo_estado, recompensa, terminado)]}

# Veamos qué pasa en el estado 324 para cada una de las 6 acciones posibles:
env.unwrapped.P[324]

# Interpretación del resultado:
# 0 (sur): ir al estado 424, recompensa -1, no termina
# 1 (norte): ir al estado 224, recompensa -1, no termina
# 2 (este): ir al estado 344, recompensa -1, no termina
# 3 (oeste): quedarse en 324 (pared), recompensa -1, no termina
# 4 (recoger): quedarse en 324, recompensa -10 (ilegal), no termina
# 5 (dejar): quedarse en 324, recompensa -10 (ilegal), no termina

{0: [(1.0, 424, -1, False)],
 1: [(1.0, 224, -1, False)],
 2: [(1.0, 344, -1, False)],
 3: [(1.0, 324, -1, False)],
 4: [(1.0, 324, -10, False)],
 5: [(1.0, 324, -10, False)]}

Este diccionario tiene la estructura {acción: [(probabilidad, próximo estado, recompensa, hecho)]}.

Algunas cosas a tener en cuenta:

* Los números del 0 al 5 corresponden a las acciones (sur, norte, este, oeste, recoger, dejar) que el taxi puede realizar en nuestro estado actual en la ilustración.
* En este entorno, la probabilidad siempre es 1.0.
* El próximo estado es el estado en el que estaríamos si tomamos la acción en este índice del diccionario.
* Todas las acciones de movimiento tienen una recompensa de -1 y las acciones de recoger/dejar tienen una recompensa de -10 en este estado en particular. Si estamos en un estado donde el taxi tiene un pasajero y está encima del destino correcto, veríamos una recompensa de 20 en la acción de dejar (5).
* done se utiliza para indicarnos cuándo hemos dejado con éxito a un pasajero en la ubicación correcta. Cada entrega exitosa es el final de un episodio.

Ten en cuenta que si nuestro agente elige explorar la acción dos (2) en este estado, estaría yendo hacia el Este, hacia una pared. El código fuente ha hecho imposible mover realmente el taxi a través de una pared, así que si el taxi elige esa acción, simplemente seguirá acumulando penalizaciones de -1, lo que afecta a la recompensa a largo plazo.


## Without Reinforcement Learning

In [8]:
# =============================================================================
# EXPERIMENTO: TAXI SIN INTELIGENCIA (ACCIONES ALEATORIAS)
# =============================================================================

# Creamos un nuevo entorno limpio
env = gym.make("Taxi-v3", render_mode="ansi")
state, info = env.reset()

# Configuramos el estado inicial al estado 324 de ejemplo
env.unwrapped.s = 324

# Variables para métricas
epochs = 0          # Contador de pasos/movimientos
penalties = 0       # Contador de penalizaciones
reward = 0          # Recompensa actual

frames = []         # Lista para guardar cada frame de la animación

terminated = False  # Flag que indica si el episodio terminó (pasajero entregado)

# Bucle principal: el taxi actúa hasta entregar al pasajero
while not terminated:
    # env.action_space.sample() elige una acción ALEATORIA (0-5)
    # Esto simula un taxi sin inteligencia que actúa al azar
    action = env.action_space.sample()
    
    # env.step(action) ejecuta la acción y devuelve:
    # - state: nuevo estado después de la acción
    # - reward: recompensa obtenida (+20 éxito, -1 movimiento, -10 acción ilegal)
    # - terminated: True si se completó el objetivo
    # - truncated: True si se alcanzó límite de tiempo
    # - info: información adicional
    state, reward, terminated, truncated, info = env.step(action)
    
    # Contamos penalizaciones (recogida/entrega ilegal)
    if reward == -10:
        penalties = penalties + 1
    
    # Guardamos el frame actual para poder animarlo después
    frames.append({
        'frame': env.render(),  # Visualización del estado
        'state': state,          # Número del estado
        'action': action,        # Acción tomada
        'reward': reward         # Recompensa recibida
    })

    epochs += 1  # Incrementamos el contador de pasos

# Mostramos resultados del experimento aleatorio
print("Timesteps taken: {}".format(epochs))        # Cuántos pasos tardó
print("Penalties incurred: {}".format(penalties))  # Cuántas penalizaciones recibió

Timesteps taken: 2831
Penalties incurred: 902


In [9]:
frames

[{'frame': '+---------+\n|\x1bR\x1b: | : :\x1bG\x1b|\n| : | : : |\n| :\x1b \x1b: : : |\n| | : | : |\n|Y| : |B: |\n+---------+\n  (North)\n',
  'state': 224,
  'action': np.int64(1),
  'reward': -1},
 {'frame': '+---------+\n|\x1bR\x1b: | : :\x1bG\x1b|\n| : | : : |\n| :\x1b \x1b: : : |\n| | : | : |\n|Y| : |B: |\n+---------+\n  (Dropoff)\n',
  'state': 224,
  'action': np.int64(5),
  'reward': -10},
 {'frame': '+---------+\n|\x1bR\x1b: | : :\x1bG\x1b|\n| : | : : |\n| : : : : |\n| |\x1b \x1b: | : |\n|Y| : |B: |\n+---------+\n  (South)\n',
  'state': 324,
  'action': np.int64(0),
  'reward': -1},
 {'frame': '+---------+\n|\x1bR\x1b: | : :\x1bG\x1b|\n| : | : : |\n| :\x1b \x1b: : : |\n| | : | : |\n|Y| : |B: |\n+---------+\n  (North)\n',
  'state': 224,
  'action': np.int64(1),
  'reward': -1},
 {'frame': '+---------+\n|\x1bR\x1b: | : :\x1bG\x1b|\n| : | : : |\n| : : : : |\n| |\x1b \x1b: | : |\n|Y| : |B: |\n+---------+\n  (South)\n',
  'state': 324,
  'action': np.int64(0),
  'reward': -1},
 {

In [10]:
def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        time.sleep(.2)

In [11]:
# =============================================================================
# FUNCIÓN PARA ANIMAR LA SECUENCIA DE ACCIONES DEL TAXI
# =============================================================================

from IPython.display import clear_output  # Para limpiar la salida en notebooks
from time import sleep  # Para pausar entre frames

def print_frames(frames):
    """
    Función que anima la secuencia de movimientos del taxi
    
    Parámetros:
    - frames: Lista de diccionarios con información de cada paso
    """
    # Iteramos sobre cada frame guardado
    for i, frame in enumerate(frames):
        clear_output(wait=True)  # Limpiamos la salida anterior
        
        # Mostramos el estado visual del entorno
        print(frame['frame'])
        
        # Mostramos información del paso actual
        print(f"Timestep: {i + 1}")           # Número de paso (empezando en 1)
        print(f"State: {frame['state']}")     # Estado numérico (0-499)
        print(f"Action: {frame['action']}")   # Acción tomada (0-5)
        print(f"Reward: {frame['reward']}")   # Recompensa recibida
        
        sleep(.2)  # Pausa de 0.2 segundos entre frames para ver la animación

# Ejecutamos la animación con los frames capturados anteriormente
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)

Timestep: 1509
State: 252
Action: 1
Reward: -1


KeyboardInterrupt: 

[algoritmo A*](https://www.datacamp.com/tutorial/a-star-algorithm)

## With Reinforcement Learning

Vamos a utilizar un algoritmo simple de aprendizaje por refuerzo llamado Q-learning, que le dará a nuestro agente algo de memoria.


Básicamente, el Q-learning permite que el agente utilice las recompensas del entorno para aprender, con el tiempo, la mejor acción a tomar en un estado dado.

En nuestro entorno Taxi, tenemos la tabla de recompensas, P, de la que el agente aprenderá. Lo hace al recibir una recompensa por tomar una acción en el estado actual, y luego actualiza un valor Q para recordar si esa acción fue beneficiosa.

Los valores almacenados en la tabla Q se llaman valores Q, y se corresponden con una combinación (estado, acción).

Un valor Q para una combinación particular de estado-acción es representativo de la "calidad" de una acción tomada desde ese estado. Mejores valores Q implican mejores posibilidades de obtener recompensas mayores.

Por ejemplo, si el taxi se enfrenta a un estado que incluye un pasajero en su ubicación actual, es muy probable que el valor Q para recoger sea más alto en comparación con otras acciones, como dejar o ir hacia el norte.


Estamos asignando (), o actualizando, el valor Q del estado y la acción actual del agente primero tomando un peso () del antiguo valor Q, y luego añadiendo el valor aprendido. El valor aprendido es una combinación de la recompensa por tomar la acción actual en el estado actual, y la recompensa máxima descontada del próximo estado en el que estaremos una vez que tomemos la acción actual.

Básicamente, estamos aprendiendo la acción adecuada a tomar en el estado actual al observar la recompensa para la combinación estado/acción actual y las máximas recompensas para el próximo estado. Esto eventualmente hará que nuestro taxi considere la ruta con las mejores recompensas concatenadas.

El valor Q de un par estado-acción es la suma de la recompensa instantánea y la recompensa futura descontada (del estado resultante). La forma en que almacenamos los valores Q para cada estado y acción es a través de una tabla Q.


### Q-table

La tabla Q es una matriz donde tenemos una fila para cada estado (500) y una columna para cada acción (6). Se inicializa primero en 0, y luego los valores se actualizan después del entrenamiento. Ten en cuenta que la tabla Q tiene las mismas dimensiones que la tabla de recompensas, pero tiene un propósito completamente diferente.


<img src="./img/q-matrix-initialized-to-learned_gQq0BFs.png" alt="drawing" width="650"/>

Desglosándolo en pasos, obtenemos:

* Inicializar la tabla Q con todos los valores en cero.
* Comenzar a explorar acciones: Para cada estado, seleccionar una de entre todas las acciones posibles para el estado actual (S).
* Viajar al siguiente estado (S') como resultado de esa acción (a).
* Para todas las acciones posibles desde el estado (S'), seleccionar aquella con el valor Q más alto.
* Actualizar los valores de la tabla Q usando la ecuación.
* Establecer el siguiente estado como el estado actual.
* Si se alcanza el estado objetivo, entonces finalizar y repetir el proceso.


Después de suficiente exploración aleatoria de acciones, los valores Q tienden a converger, sirviendo a nuestro agente como una función de valor de acción que puede explotar para elegir la acción más óptima a partir de un estado dado.

Existe un compromiso entre la exploración (elegir una acción al azar) y la explotación (elegir acciones basadas en los valores Q ya aprendidos). Queremos evitar que la acción siempre tome la misma ruta, y posiblemente sobreajuste, así que introduciremos otro parámetro llamado "epsilon" para atender a esto durante el entrenamiento.

En lugar de simplemente seleccionar la mejor acción aprendida con valor Q, a veces favoreceremos la exploración adicional del espacio de acciones. Un valor epsilon más bajo resulta en episodios con más penalizaciones (en promedio), lo que es obvio porque estamos explorando y tomando decisiones al azar.


## Vamos a construirlo!

In [12]:
# =============================================================================
# INICIALIZACIÓN DE LA TABLA Q (Q-TABLE)
# =============================================================================

import numpy as np  # NumPy para operaciones con arrays

# Creamos la Q-table: una matriz de CEROS
# Dimensiones:
# - Filas: 500 (un estado por fila, correspondiente a cada estado del entorno)
# - Columnas: 6 (una acción por columna: sur, norte, este, oeste, recoger, dejar)
# 
# Cada celda [estado, acción] contendrá el valor Q, que representa:
# "Qué tan bueno es tomar esa acción en ese estado"
# 
# Al inicio todos los valores son 0 porque el agente no sabe nada
q_table = np.zeros([env.observation_space.n, env.action_space.n])

# Mostramos la tabla Q inicial
q_table

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]], shape=(500, 6))

In [13]:
# Verificamos las dimensiones de la Q-table
# Debería ser (500, 6): 500 estados x 6 acciones
q_table.shape

(500, 6)

In [14]:
# Verificamos el número total de elementos en la Q-table
# 500 estados * 6 acciones = 3000 valores Q para aprender
q_table.size

3000

Ahora podemos crear el algoritmo de entrenamiento que actualizará esta tabla Q a medida que el agente explore el entorno a lo largo de miles de episodios.

En la primera parte de while not done, decidimos si elegir una acción al azar o explotar los valores Q ya calculados. Esto se hace simplemente utilizando el valor de epsilon y comparándolo con la función random.uniform(0, 1), que devuelve un número arbitrario entre 0 y 1.

Ejecutamos la acción elegida en el entorno para obtener el próximo estado y la recompensa por realizar la acción. Después de eso, calculamos el valor Q máximo para las acciones correspondientes al próximo estado, y con eso, podemos actualizar fácilmente nuestro valor Q al nuevo valor_q:


In [15]:
q_table

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]], shape=(500, 6))

In [16]:
# =============================================================================
# ENTRENAMIENTO DEL AGENTE CON Q-LEARNING
# =============================================================================

# %%time  # Comando mágico de Jupyter para medir tiempo de ejecución (descomentarlo si deseas)

import random  # Para generar números aleatorios
from IPython.display import clear_output  # Para limpiar la salida durante entrenamiento

# Reinicializamos la Q-table a ceros
q_table = np.zeros([env.observation_space.n, env.action_space.n])

# ========== HIPERPARÁMETROS DEL ALGORITMO Q-LEARNING ==========
# Alpha (α): Tasa de aprendizaje - qué tanto confiamos en nueva información vs vieja
# Un valor de 0.1 significa: 10% nueva info, 90% info antigua
alpha = 0.1

# Gamma (γ): Factor de descuento - importancia de recompensas futuras vs inmediatas
# Un valor de 0.6 significa: las recompensas futuras valen 60% de las inmediatas
gamma = 0.6

# Epsilon (ε): Probabilidad de exploración vs explotación
# Un valor de 0.1 significa: 10% exploración aleatoria, 90% usar conocimiento
epsilon = 0.1

# Listas para guardar métricas (opcional, no se usan actualmente)
all_epochs = []
all_penalties = []

# ========== BUCLE PRINCIPAL DE ENTRENAMIENTO ==========
# Entrenaremos por 50,000 episodios (cada episodio = recoger y dejar un pasajero)
for i in range(1, 50001):
    # Reseteamos el entorno para un nuevo episodio
    state, info = env.reset()

    # Variables para métricas de este episodio
    epochs = 0      # Contador de pasos
    penalties = 0   # Contador de penalizaciones
    reward = 0      # Recompensa actual
    terminated = False  # Flag de finalización

    # Bucle del episodio: hasta que el pasajero sea entregado
    while not terminated:
        # ========== DECISIÓN: EXPLORAR O EXPLOTAR ==========
        if random.uniform(0, 1) < epsilon:
            # EXPLORACIÓN: Elegimos una acción aleatoria (10% del tiempo)
            action = env.action_space.sample()
        else:
            # EXPLOTACIÓN: Elegimos la mejor acción según la Q-table (90% del tiempo)
            # np.argmax() encuentra el índice de la acción con mayor valor Q
            action = np.argmax(q_table[state])

        # Ejecutamos la acción en el entorno
        next_state, reward, terminated, truncated, info = env.step(action)

        # ========== ACTUALIZACIÓN DE LA Q-TABLE (ECUACIÓN DE BELLMAN) ==========
        # Obtenemos el valor Q actual para esta combinación estado-acción
        old_value = q_table[state, action]
        
        # Encontramos el mejor valor Q posible en el próximo estado
        # Esto representa la mejor recompensa futura esperada
        next_max = np.max(q_table[next_state])
        
        # Calculamos el nuevo valor Q usando la ecuación de Q-learning:
        # nuevo_Q = (1-α)*viejo_Q + α*(recompensa_actual + γ*mejor_Q_futuro)
        # Esto combina la estimación antigua con la nueva información
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        
        # Actualizamos la Q-table con el nuevo valor
        q_table[state, action] = new_value

        # Contamos penalizaciones para estadísticas
        if reward == -10:
            penalties += 1

        # Avanzamos al siguiente estado
        state = next_state
        epochs += 1

    # Cada 100 episodios mostramos el progreso
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\\n")

Episode: 50000
Training finished.\n


Ahora que la tabla Q se ha establecido durante 100,000 episodios, veamos cuáles son los valores Q en el estado de nuestra ilustración:

In [17]:
# =============================================================================
# INSPECCIÓN DE LA TABLA DE TRANSICIONES PARA EL ESTADO 324
# =============================================================================

# Recordamos qué pasa en el estado 324 según la dinámica del entorno
# Esto nos ayudará a comparar con lo que aprendió el agente en la Q-table
env.unwrapped.P[324]

{0: [(1.0, 424, -1, False)],
 1: [(1.0, 224, -1, False)],
 2: [(1.0, 344, -1, False)],
 3: [(1.0, 324, -1, False)],
 4: [(1.0, 324, -10, False)],
 5: [(1.0, 324, -10, False)]}

In [18]:
# Verificamos el espacio de acciones del entorno
# Debería mostrar Discrete(6) indicando 6 acciones posibles
env.action_space

Discrete(6)

In [19]:
# =============================================================================
# ANÁLISIS DE LOS VALORES Q APRENDIDOS PARA EL ESTADO 324
# =============================================================================

# Accedemos a la fila 324 de la Q-table para ver qué aprendió el agente
# Cada valor representa "qué tan buena" es cada acción en este estado
# np.round(..., 4) redondea a 4 decimales para mejor legibilidad

# Los valores Q son negativos porque el taxi acumula recompensas negativas (-1 por movimiento)
# hasta llegar al objetivo (+20). El valor Q representa la recompensa acumulada esperada.

# Interpretación:
# - Valores más altos (menos negativos) = mejores acciones
# - Valores muy bajos (muy negativos) = malas acciones (como recoger/dejar en lugar incorrecto)

np.round(q_table[324], 4)

array([ -2.4917,  -2.4894,  -2.4894,  -2.4916,  -9.2704, -10.0256])

1. sur
2. norte
3. este
4. oeste
5. recoger
6. dejar

In [20]:
# =============================================================================
# IDENTIFICAR LA MEJOR ACCIÓN APRENDIDA PARA EL ESTADO 324
# =============================================================================

# np.argmax() encuentra el índice (0-5) del valor más alto en la fila 324
# Este índice corresponde a la mejor acción que el agente aprendió para este estado

# Recordatorio de acciones:
# 0 = sur, 1 = norte, 2 = este, 3 = oeste, 4 = recoger, 5 = dejar

np.argmax(q_table[324])

np.int64(2)

El valor máximo de Q es "north" o "east" (-2.489), ¡así que parece que Q-learning ha aprendido efectivamente la mejor acción a tomar en el estado de nuestra ilustración!


### Evaluate

Vamos a evaluar el rendimiento de nuestro agente. Ya no necesitamos explorar acciones más, así que ahora la siguiente acción siempre se selecciona usando el mejor valor Q:


In [21]:
# =============================================================================
# EVALUACIÓN DEL AGENTE ENTRENADO (CON Q-LEARNING)
# =============================================================================

# Ahora evaluaremos qué tan bien funciona el agente DESPUÉS del entrenamiento
# El agente ya no explora, solo explota su conocimiento (usa la Q-table)

total_epochs = 0      # Suma total de pasos de todos los episodios
total_penalties = 0   # Suma total de penalizaciones
episodes = 10         # Número de episodios de prueba
frames = []           # Para guardar animación

# Probamos el agente en 10 episodios diferentes
for _ in range(episodes):
    # Comenzamos un nuevo episodio con estado aleatorio
    state, info = env.reset()
    
    # NOTA: Las siguientes líneas están comentadas pero permitirían
    # establecer un estado inicial específico para pruebas consistentes
    # state = env.unwrapped.encode(3, 1, 2, 0)
    # env.unwrapped.s = state
    
    # Variables para este episodio
    epochs = 0
    penalties = 0
    reward = 0
    terminated = False
    actions = []  # Lista para guardar la secuencia de acciones

    # Bucle del episodio
    while not terminated:
        # ========== EXPLOTACIÓN PURA (SIN EXPLORACIÓN) ==========
        # El agente SIEMPRE elige la mejor acción según la Q-table
        # Ya no hay aleatoriedad, usa solo lo que aprendió
        action = np.argmax(q_table[state])
        actions.append(action)
        
        # Ejecutamos la acción elegida
        state, reward, terminated, truncated, info = env.step(action)
        
        # Guardamos el frame para animación posterior
        frames.append({
            'frame': env.render(),
            'state': state,
            'action': action,
            'reward': reward
        })
        
        # Contamos penalizaciones (idealmente debería ser 0)
        if reward == -10:
            penalties += 1

        epochs += 1
    
    print("Finalizada partida", _)
    
    # Acumulamos métricas
    total_penalties += penalties
    total_epochs += epochs

# ========== RESULTADOS DE LA EVALUACIÓN ==========
print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")  # Debería ser bajo (~12-15)
print(f"Average penalties per episode: {total_penalties / episodes}")  # Debería ser 0

Finalizada partida 0
Finalizada partida 1
Finalizada partida 2
Finalizada partida 3
Finalizada partida 4
Finalizada partida 5
Finalizada partida 6
Finalizada partida 7
Finalizada partida 8
Finalizada partida 9
Results after 10 episodes:
Average timesteps per episode: 13.3
Average penalties per episode: 0.0


In [22]:
# Animamos la última evaluación del agente entrenado
# Deberías ver movimientos inteligentes y eficientes hacia el objetivo
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 133
State: 0
Action: 5
Reward: 20


In [23]:
# Mostramos la secuencia de acciones del último episodio
# Esto muestra la "estrategia" que siguió el agente
# Cada número representa: 0=sur, 1=norte, 2=este, 3=oeste, 4=recoger, 5=dejar
actions

[np.int64(0),
 np.int64(0),
 np.int64(3),
 np.int64(3),
 np.int64(0),
 np.int64(0),
 np.int64(4),
 np.int64(1),
 np.int64(1),
 np.int64(1),
 np.int64(1),
 np.int64(5)]

Podemos ver en la evaluación que el rendimiento del agente mejoró significativamente y no incurrió en penalizaciones, lo que significa que realizó las acciones correctas de recogida/dejar con 100 pasajeros diferentes.


In [24]:
# Genera una acción aleatoria del espacio de acciones
# Esto es equivalente a elegir un número aleatorio entre 0 y 5
env.action_space.sample()

np.int64(0)

In [25]:
# =============================================================================
# EVALUACIÓN DEL AGENTE SIN ENTRENAMIENTO (ACCIONES ALEATORIAS)
# =============================================================================

# Para comparar, evaluamos un agente que actúa COMPLETAMENTE al azar
# Esto nos muestra la importancia del aprendizaje por refuerzo

total_epochs = 0
total_penalties = 0
episodes = 100  # Usamos más episodios para obtener un promedio confiable

for _ in range(episodes):
    # Reiniciamos el entorno
    state, info = env.reset()
    
    # Establecemos un estado inicial fijo para comparación justa
    state = env.unwrapped.encode(3, 1, 2, 0)
    env.unwrapped.s = state
    
    # Variables del episodio
    epochs = 0
    penalties = 0
    reward = 0
    terminated = False
    actions = []

    # Bucle del episodio
    while not terminated:
        # ========== ACCIÓN COMPLETAMENTE ALEATORIA ==========
        # El agente NO usa conocimiento, solo elige al azar
        action = env.action_space.sample()
        actions.append(action)
        
        # Ejecutamos la acción
        state, reward, terminated, truncated, info = env.step(action)

        # Contamos penalizaciones
        if reward == -10:
            penalties += 1

        epochs += 1

    # Acumulamos métricas
    total_penalties += penalties
    total_epochs += epochs

# ========== COMPARACIÓN DE RESULTADOS ==========
print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")  # Será MUCHO mayor (~2000+)
print(f"Average penalties per episode: {total_penalties / episodes}")  # Será alto (~700+)

# CONCLUSIÓN: El agente aleatorio tarda ~150x más y comete ~1000x más errores
# que el agente entrenado. Esto demuestra el poder del Q-Learning.

Results after 100 episodes:
Average timesteps per episode: 1982.39
Average penalties per episode: 639.69


# SARSA (State-Action-Reward-State-Action)

SARSA es un algoritmo específico de aprendizaje por refuerzo que se utiliza para actualizar los valores de acción en función de la observación del siguiente estado y acción, además de la recompensa actual. En SARSA, se elige una acción (A) en un estado (S), se observa el siguiente estado (S') y se elige una nueva acción (A') basada en una política de toma de decisiones (que puede ser ε-greedy, por ejemplo). Luego, se actualizan los valores de acción utilizando la recompensa recibida y el valor de acción del siguiente estado y acción.



La principal diferencia entre Q-Table y SARSA radica en cómo se actualizan los valores de acción.
En la Q-Table, los valores se actualizan considerando el máximo valor de acción posible en el siguiente estado, independientemente de la acción tomada. En cambio, SARSA actualiza los valores de acción utilizando la acción real tomada en el siguiente estado.
Por lo tanto, mientras que Q-Table es un método off-policy (actualiza los valores de acción considerando la mejor acción posible), SARSA es un método on-policy (actualiza los valores de acción considerando la acción real tomada).


Q-Table:
- Ventajas: Es simple de entender e implementar en entornos con un número limitado de estados y acciones.
- Inconvenientes: No es escalable para entornos con un gran número de estados y acciones debido a la necesidad de almacenar y actualizar una tabla grande de valores de acción.

SARSA:
- Ventajas: Es más eficiente en términos de memoria y puede escalar mejor a entornos con un gran número de estados y acciones.
- Inconvenientes: Puede ser más difícil de implementar y entender en comparación con Q-Table debido a la necesidad de seguir una política de toma de decisiones y actualizar los valores de acción de manera adecuada.

In [26]:
# =============================================================================
# ENTRENAMIENTO CON SARSA (State-Action-Reward-State-Action)
# =============================================================================

# SARSA es similar a Q-Learning pero con una diferencia clave:
# - Q-Learning es "off-policy": actualiza usando la MEJOR acción posible del siguiente estado
# - SARSA es "on-policy": actualiza usando la acción que REALMENTE tomará en el siguiente estado

# Reinicializamos la Q-table
q_table = np.zeros([env.observation_space.n, env.action_space.n])

# Hiperparámetros (los mismos que en Q-Learning para comparación)
alpha = 0.1    # Tasa de aprendizaje
gamma = 0.6    # Factor de descuento
epsilon = 0.1  # Probabilidad de exploración

# Métricas
all_epochs = []
all_penalties = []

# Entrenamiento por 100,000 episodios
for i in range(1, 100001):
    # Reiniciamos el entorno
    state, info = env.reset()
    
    # ========== DIFERENCIA CLAVE: ELEGIMOS LA PRIMERA ACCIÓN ANTES DEL BUCLE ==========
    # En SARSA necesitamos la acción inicial para poder actualizar con la acción real del siguiente paso
    action = env.action_space.sample() if random.uniform(0, 1) < epsilon else np.argmax(q_table[state])

    epochs = 0
    penalties = 0
    reward = 0
    terminated = False
    
    while not terminated:
        # Ejecutamos la acción actual
        next_state, reward, terminated, truncated, info = env.step(action)
        
        # ========== ELEGIMOS LA PRÓXIMA ACCIÓN (que realmente tomaremos) ==========
        next_action = env.action_space.sample() if random.uniform(0, 1) < epsilon else np.argmax(q_table[next_state])

        # ========== ACTUALIZACIÓN SARSA ==========
        old_value = q_table[state, action]
        
        # DIFERENCIA PRINCIPAL: En lugar de usar max(Q[next_state]), 
        # usamos Q[next_state, next_action] (la acción que realmente tomaremos)
        next_value = q_table[next_state, next_action]
        
        # Ecuación de actualización SARSA:
        # Q(s,a) = Q(s,a) + α * [r + γ*Q(s',a') - Q(s,a)]
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_value)
        q_table[state, action] = new_value

        # Contamos penalizaciones
        if reward == -10:
            penalties += 1

        # ========== AVANZAMOS AL SIGUIENTE ESTADO Y ACCIÓN ==========
        # Esto es clave en SARSA: usamos la acción que ya elegimos
        state = next_state
        action = next_action
        epochs += 1

    # Mostramos progreso cada 100 episodios
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\\n")

# COMPARACIÓN Q-Learning vs SARSA:
# - Q-Learning: Más agresivo, aprende la política óptima incluso si explora
# - SARSA: Más conservador, aprende la política que realmente sigue (incluyendo exploración)
# - En la práctica, Q-Learning suele converger más rápido en entornos determinísticos

Episode: 100000
Training finished.\n


In [ ]:
# # Q-Learning
# action = np.argmax(q_table[state]) # Exploit learned values

# # SARSA
# next_action = np.argmax(q_table[next_state]) # Choose next action

## Alpha (α):
El parámetro alpha controla la tasa de aprendizaje en los algoritmos de aprendizaje por refuerzo. Es una medida de cuánto confiamos en las nuevas actualizaciones de los valores de acción en comparación con los valores existentes.

- Si alpha es alto, damos más peso a las nuevas recompensas para actualizar los valores de acción.
- Si alpha es bajo, damos más peso a los valores de acción existentes y aprendemos más lentamente.

Ejemplo: Imagina que estás aprendiendo a jugar al ajedrez. Si tienes un alpha alto, estarías cambiando tus estrategias rápidamente después de cada partida. Si tienes un alpha bajo, estarías más inclinado a mantener tus estrategias existentes durante más tiempo, incluso si no te están dando buenos resultados.

## Gamma (γ):
El parámetro gamma es el factor de descuento en los algoritmos de aprendizaje por refuerzo. Controla cuánto valoramos las recompensas futuras en comparación con las recompensas inmediatas.
- Un gamma cercano a 1 significa que valoramos mucho las recompensas futuras.
- Un gamma cercano a 0 significa que solo valoramos las recompensas inmediatas.

Ejemplo: Imagina que estás decidiendo si estudiar para un examen o salir con tus amigos. Si tienes un gamma alto, estarías más inclinado a estudiar porque valoras mucho las recompensas futuras (buenas calificaciones). Si tienes un gamma bajo, estarías más inclinado a salir con tus amigos porque solo valoras la recompensa inmediata (diversión).

## Epsilon (ε):
El parámetro epsilon controla la exploración frente a la explotación en los algoritmos de aprendizaje por refuerzo. Determina la probabilidad de elegir una acción al azar en lugar de la acción óptima según los valores de acción actuales.
- Un epsilon alto significa que somos más propensos a explorar nuevas acciones.
- Un epsilon bajo significa que somos más propensos a explotar las acciones conocidas.

Ejemplo: Imagina que estás decidiendo qué película ver en Netflix. Si tienes un epsilon alto, es más probable que explores nuevas películas en lugar de ver tus favoritas. Si tienes un epsilon bajo, es más probable que veas tus películas favoritas una y otra vez sin explorar nuevas opciones.